# 上网业务——网络覆盖与信号强度

In [1]:
from sklearnex import patch_sklearn, config_context
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier,RandomForestClassifier,StackingClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,accuracy_score
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import missingno as msno
from time import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
plt.rcParams['font.sans-serif']=['SimHei']

##  预处理数据导入

In [3]:
train = pd.read_excel("./output/上网训练集.xlsx")

In [4]:
train_data = pd.DataFrame.copy(train, deep=True)
train_data

,用户,手机上网整体满意度,网络覆盖与信号强度,手机上网速度,手机上网稳定性,居民小区,办公室,高校,商业街,地铁,...,是否5G网络客户,当月GPRS资源使用量（GB）,当月MOU,客户星级标识,终端品牌_0,终端品牌_其它品牌,终端品牌_华为,终端品牌_小米科技,终端品牌_步步高,终端品牌_苹果
0,用户1,8,7,7,7,0,0,0,0,0,...,0,0.0313,54,3,0,0,0,1,0,0
1,用户2,9,9,8,9,0,0,0,0,1,...,0,5.7303,172,1,0,0,1,0,0,0
2,用户3,4,3,3,3,1,1,1,1,1,...,0,35.2103,206,3,0,0,0,0,0,1
3,用户4,9,8,8,7,0,0,0,0,1,...,0,74.8815,1930,3,0,0,0,0,0,1
4,用户5,10,10,10,10,1,0,1,0,0,...,0,44.5678,18,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7015,用户7016,10,10,10,10,0,0,0,0,0,...,1,7.3362,377,5,0,0,1,0,0,0
7016,用户7017,9,9,9,9,0,0,0,0,1,...,0,12.4199,208,5,0,0,1,0,0,0
7017,用户7018,10,10,10,10,0,0,0,0,0,...,0,23.9026,444,5,0,0,1,0,0,0
7018,用户7019,5,5,5,5,1,1,0,1,0,...,0,15.9536,392,5,0,0,1,0,0,0


In [6]:
train_data=train_data.drop(['用户'],axis=1)
train_data

,网络覆盖与信号强度,居民小区,办公室,高校,商业街,地铁,农村,高铁,其他，请注明,网络信号差/没有信号,...,是否5G网络客户,当月GPRS资源使用量（GB）,当月MOU,客户星级标识,终端品牌_0,终端品牌_其它品牌,终端品牌_华为,终端品牌_小米科技,终端品牌_步步高,终端品牌_苹果
0,7,0,0,0,0,0,0,0,0,0,...,0,0.0313,54,3,0,0,0,1,0,0
1,9,0,0,0,0,1,0,0,0,1,...,0,5.7303,172,1,0,0,1,0,0,0
2,3,1,1,1,1,1,1,1,0,1,...,0,35.2103,206,3,0,0,0,0,0,1
3,8,0,0,0,0,1,0,0,0,0,...,0,74.8815,1930,3,0,0,0,0,0,1
4,10,1,0,1,0,0,0,0,0,1,...,0,44.5678,18,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7015,10,0,0,0,0,0,0,0,0,0,...,1,7.3362,377,5,0,0,1,0,0,0
7016,9,0,0,0,0,1,1,0,0,1,...,0,12.4199,208,5,0,0,1,0,0,0
7017,10,0,0,0,0,0,0,0,0,0,...,0,23.9026,444,5,0,0,1,0,0,0
7018,5,1,1,0,1,0,0,1,0,1,...,0,15.9536,392,5,0,0,1,0,0,0


## 数据划分

In [12]:
x = train_data.drop(['网络覆盖与信号强度'],axis=1)
x

,网络信号差/没有信号,手机上网速度慢,打开网页或APP图片慢,居民小区,上网过程中网络时断时续或时快时慢,显示有信号上不了网,看视频卡顿,办公室,地铁,打游戏延时大,...,全部网页或APP都慢,手机支付较慢,全部都卡顿,微信,商业街,终端品牌_苹果,高铁,淘宝,王者荣耀,抖音
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,0,1,1,1,0,1,0,...,0,1,0,0,0,0,0,0,0,0
2,1,1,1,1,0,0,0,1,1,0,...,1,0,0,0,1,1,1,0,0,0
3,0,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7015,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7016,1,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7017,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7018,1,1,1,1,1,1,0,1,0,0,...,0,0,0,1,1,0,1,1,0,0


In [13]:
y = train_data["网络覆盖与信号强度"]
y

0        7
1        9
2        3
3        8
4       10
        ..
7015    10
7016     9
7017    10
7018     5
7019     1
Name: 网络覆盖与信号强度, Length: 7020, dtype: int64

In [14]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

## 模型训练

In [15]:
rg = RidgeClassifier()
parameters = {
    "alpha":[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0],
    "normalize":[True,False]
}

rg_cv = GridSearchCV(rg, parameters, cv=5)
rg_cv.fit(x_train,y_train)
rg_cv.best_estimator_

RidgeClassifier(alpha=0.1, normalize=True)

In [16]:
dt = DecisionTreeClassifier()
parameters = {
    "criterion":['gini', 'entropy','log_loss'],
    "max_depth":[2,4,8,16,32],
    "min_samples_leaf":[2,4,8,16,32],
    "min_samples_split":[2,4,8,16,32]
}

dt_cv = GridSearchCV(dt, parameters, cv=5)
dt_cv.fit(x_train,y_train)
dt_cv.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=4, min_samples_leaf=32)

In [17]:
rf = RandomForestClassifier()
parameters = {
    "max_depth":[2,4,8,16,32],
    "n_estimators":[5,50,250,500]
}

rf_cv = GridSearchCV(rf, parameters, cv=5)
rf_cv.fit(x_train,y_train)
rf_cv.best_estimator_

RandomForestClassifier(max_depth=4, n_estimators=250)

In [18]:
ada = AdaBoostClassifier()
parameters = {
    "learning_rate":[0.01,0.1,1.0,10,100],
    "n_estimators":[5,50,250,500]
}

ada_cv = GridSearchCV(ada, parameters, cv=5)
ada_cv.fit(x_train,y_train)
ada_cv.best_estimator_

AdaBoostClassifier(learning_rate=0.01, n_estimators=500)

In [ ]:
# gb = GradientBoostingClassifier()
# parameters = {
#     "learning_rate":[0.01,0.1,1.0,10,100],
#     "max_depth":[2,4,8,16,32],
#     "n_estimators":[5,50,250,500]
# }

# gb_cv = GridSearchCV(gb, parameters, cv=5)
# gb_cv.fit(x_train,y_train)
# gb_cv.best_estimator_

In [19]:
xgb = XGBClassifier()
parameters = {
    "n_estimators":[5,50,250,500],
    "max_depth":[2,4,8,16,32],
    "learning_rate":[0.01,0.1,1.0,10,100]
}

xgb_cv = GridSearchCV(xgb, parameters, cv=5)
xgb_cv.fit(x_train,y_train)
xgb_cv.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=50, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [20]:
svr = SVC()
parameters = {
    "C":[0.001,0.01,0.1,1.0,10,100,1000],
    "kernel":["linear","poly","rbf","sigmoid"]
}

svr_cv = GridSearchCV(svr, parameters, cv=5)
svr_cv.fit(x_train,y_train)
svr_cv.best_estimator_

SVC(C=0.1, kernel='poly')

In [21]:
mlp = MLPClassifier()
parameters = {
    "activation":["identity","logistic","tanh","relu"],
    "hidden_layer_sizes":[(10,),(50,),(100,)],
    "learning_rate":["constant","invscaling","adaptive"],
    "solver":["lbfgs","sgd","adam"]
}

mlp_cv = GridSearchCV(mlp, parameters, cv=5)
mlp_cv.fit(x_train, y_train)
mlp_cv.best_estimator_

MLPClassifier(activation='tanh', hidden_layer_sizes=(10,),
              learning_rate='adaptive', solver='sgd')

In [22]:
estimators = [("rg",rg_cv.best_estimator_),
              ("dt",dt_cv.best_estimator_),
              ("rf",rf_cv.best_estimator_),
              ("ada",ada_cv.best_estimator_),
#               ("gb",gb_cv.best_estimator_),
              ("xgb",xgb_cv.best_estimator_),
              ("svr",svr_cv.best_estimator_),
              ("mlp",mlp_cv.best_estimator_)]

sr = StackingClassifier(estimators=estimators)
parameters = {
    "passthrough":[True,False]
}

sr_cv = GridSearchCV(sr, parameters, cv=5)
sr_cv.fit(x_train,y_train)
sr_cv.best_estimator_

StackingClassifier(estimators=[('rg',
                                RidgeClassifier(alpha=0.1, normalize=True)),
                               ('dt',
                                DecisionTreeClassifier(criterion='entropy',
                                                       max_depth=4,
                                                       min_samples_leaf=32)),
                               ('rf',
                                RandomForestClassifier(max_depth=4,
                                                       n_estimators=250)),
                               ('ada',
                                AdaBoostClassifier(learning_rate=0.01,
                                                   n_estimators=500)),
                               ('xgb',
                                XGBClassifier(base_score=0.5, booster='gbtree',
                                              colsample_...
                                              n_estimators=50, n_jobs=0,
           

In [23]:
models = [rg_cv.best_estimator_,
          dt_cv.best_estimator_,
          rf_cv.best_estimator_,
          ada_cv.best_estimator_,
#           gb_cv.best_estimator_,
          xgb_cv.best_estimator_,
          svr_cv.best_estimator_,
          mlp_cv.best_estimator_,
          sr_cv.best_estimator_]

In [24]:
val_set = pd.DataFrame()
    
for i in models:
        start = time()
        pred = i.predict(x_val)
        end = time()        
        temp = pd.DataFrame(
                {   
                    "Acc":("%0.3f" % (accuracy_score(y_val,pred))),
                    "MAE":("%0.3f" % (mean_absolute_error(y_val,pred))),
                    "MSE":("%0.3f" % (mean_squared_error(y_val,pred))),
                    "Latency":("%0.1fms" % ((end-start)*1000))
                }, index=[(str(i).split("(")[0])]
        )
        val_set = pd.concat([val_set,temp])
val_set

,Acc,MAE,MSE,Latency
RidgeClassifier,0.385,1.922,8.983,3.0ms
DecisionTreeClassifier,0.365,2.016,9.422,1.0ms
RandomForestClassifier,0.380,1.898,8.709,4.0ms
AdaBoostClassifier,0.390,1.875,8.600,286.3ms
XGBClassifier,0.385,1.821,8.151,7.9ms
SVC,0.380,1.839,8.182,76.8ms
MLPClassifier,0.389,1.754,7.665,2.0ms
StackingClassifier,0.387,1.766,7.706,370.0ms


## 预测（）

In [25]:
test = pd.read_excel("./output/上网测试集.xlsx")

In [26]:
test_data = pd.DataFrame.copy(test, deep=True)
test_data

,用户id,居民小区,办公室,高校,商业街,地铁,农村,高铁,其他，请注明,网络信号差/没有信号,...,GPRS总流量（KB）,是否5G网络客户,客户星级标识,是否不限量套餐到达用户,终端品牌_0,终端品牌_其它品牌,终端品牌_华为,终端品牌_小米科技,终端品牌_步步高,终端品牌_苹果
0,1,1,0,0,0,0,1,1,0,0,...,2215643,1,8,0,0,0,0,0,1,0
1,2,0,0,0,0,0,1,0,0,1,...,23273941,0,8,0,0,0,1,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,2088114,1,5,0,0,0,1,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,3691155,0,5,0,0,0,0,0,0,1
4,5,1,0,0,0,0,0,0,0,1,...,40765275,1,5,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,1606,0,0,0,0,0,0,0,0,0,...,24118010,1,8,0,0,0,1,0,0,0
1606,1607,1,0,0,0,0,0,0,0,0,...,11553961,1,5,0,0,0,1,0,0,0
1607,1608,1,1,0,0,0,0,0,0,1,...,2308312,1,4,0,0,0,0,1,0,0
1608,1609,1,0,0,0,0,0,0,0,0,...,11508141,0,8,0,0,0,1,0,0,0


In [27]:
test_col = col.drop('网络覆盖与信号强度')

In [28]:
test_data = test_data.loc[:, test_col]
test_data

,网络信号差/没有信号,手机上网速度慢,打开网页或APP图片慢,居民小区,上网过程中网络时断时续或时快时慢,显示有信号上不了网,看视频卡顿,办公室,地铁,打游戏延时大,...,全部网页或APP都慢,手机支付较慢,全部都卡顿,微信,商业街,终端品牌_苹果,高铁,淘宝,王者荣耀,抖音
0,0,1,1,1,1,0,1,0,0,0,...,0,0,0,1,0,0,1,1,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1,1,1,1,1,1,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1605,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1606,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1607,1,1,1,1,1,1,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1608,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
y_pred_test = ada_cv.best_estimator_.predict(test_data)
output = pd.DataFrame({'Id': test['用户id'], '手机上网网络覆盖与信号强度': np.round(y_pred_test, 0)})
output.to_excel('./output/result-手机上网网络覆盖与信号强度.xlsx', index=False)